In [3]:
!pip install overrides
!pip install tqdm

  Created wheel for overrides: filename=overrides-3.1.0-cp36-none-any.whl size=10174 sha256=8656315d41e56746793909360928de41fba431aac651c3b0e27fc890fc9ff286
  Stored in directory: /root/.cache/pip/wheels/5c/24/13/6ef8600e6f147c95e595f1289a86a3cc82ed65df57582c65a9
Successfully built overrides


In [1]:
! wget https://dwz.cn/ijPVPQhz

--2020-11-20 07:07:28--  https://dwz.cn/ijPVPQhz
Resolving dwz.cn (dwz.cn)... 182.61.200.113
Connecting to dwz.cn (dwz.cn)|182.61.200.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://static.leiphone.com/AI%E7%A0%94%E4%B9%A0%E7%A4%BE_%E9%B8%9F%E7%B1%BB%E8%AF%86%E5%88%AB%E6%AF%94%E8%B5%9B%E6%95%B0%E6%8D%AE%E9%9B%86.rar [following]
--2020-11-20 07:07:31--  https://static.leiphone.com/AI%E7%A0%94%E4%B9%A0%E7%A4%BE_%E9%B8%9F%E7%B1%BB%E8%AF%86%E5%88%AB%E6%AF%94%E8%B5%9B%E6%95%B0%E6%8D%AE%E9%9B%86.rar
Resolving static.leiphone.com (static.leiphone.com)... 47.246.22.227, 47.246.22.229, 47.246.22.228, ...
Connecting to static.leiphone.com (static.leiphone.com)|47.246.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320892729 (306M) [application/x-rar-compressed]
Saving to: ‘ijPVPQhz’

ijPVPQhz            100%[===================>] 306.03M  35.6MB/s    in 8.9s    

2020-11-20 07:07:42 (34.5 MB/s) - ‘ijPVPQhz’ saved [320892

In [ ]:
!unrar x ijPVPQhz


In [4]:
import torch
import pandas as pd
import torch.nn as nn
from torchvision.datasets.folder import accimage_loader, pil_loader
import os
from torchvision import transforms
import torchvision
import torch.nn.functional as F
from typing import Tuple

from torch.nn import Module
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader
from overrides import overrides
from tqdm import tqdm
from PIL import Image
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [18]:
from torch.nn import Module
import torchvision.models as models
class BilinearModel(nn.Module):
    """Load model with pretrained weights and initialise new layers."""

    def __init__(self, num_classes: int = 200) -> None:
        """Load pretrained model, set new layers with specified number of layers."""
        super(BilinearModel, self).__init__()
        model: nn.Module = models.vgg16(pretrained=True)
        self.features: nn.Module = nn.Sequential(*list(model.features)[:-1])
        self.classifier: nn.Module = nn.Linear(512 ** 2, num_classes)
        self.dropout: nn.Module = nn.Dropout(0.5)
        self.softmax: nn.Module = nn.LogSoftmax(dim=1)
        nn.init.kaiming_normal_(self.classifier.weight.data)
        if self.classifier.bias is not None:
            nn.init.constant_(self.classifier.bias.data, val=0)

    @overrides
    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """Extract input features, perform bilinear transform, project to # of classes and return."""
        outputs: torch.Tensor = self.features(inputs)               # extract features from pretrained base
        outputs = outputs.view(-1, 512, 28 ** 2)                    # reshape to batchsize * 512 * 28 ** 2
        outputs = self.dropout(outputs)
        outputs = torch.bmm(outputs, outputs.permute(0, 2, 1))      # bilinear product
        outputs = torch.div(outputs, 28 ** 2)                       # divide by 196 to normalize
        outputs = outputs.view(-1, 512 ** 2)                        # reshape to batchsize * 512 * 512
        outputs = torch.sign(outputs) * torch.sqrt(outputs + 1e-5)  # signed square root normalization
        outputs = nn.functional.normalize(outputs, p=2, dim=1)      # l2 normalization
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        # outputs = self.softmax(outputs)
        return outputs


In [7]:
"""Trainer class to abstract rudimentary training loop."""

from typing import Tuple

import torch
from torch.nn import Module
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader

from tqdm import tqdm


class Trainer(object):
    """Trainer class to abstract rudimentary training loop."""

    def __init__(
            self,
            model: Module,
            criterion: Module,
            optimizer: Optimizer,
            device: torch.device) -> None:
        """Set trainer class with model, criterion, optimizer. (Data is passed to train/eval)."""
        super(Trainer, self).__init__()
        self.model: Module = model
        self.criterion: Module = criterion
        self.optimizer: Optimizer = optimizer
        self.device: torch.device = device

    def train(self, loader: DataLoader) -> Tuple[float, float]:
        """Train model using batches from loader and return accuracy and loss."""
        total_loss, total_acc = 0.0, 0.0
        self.model.train()
        try:
            with tqdm(enumerate(loader), total=len(loader), desc='Training') as proc:
                for _, (inputs, targets) in proc:
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, targets)
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
                    _, predicted = torch.max(outputs, 1)
                    total_loss += loss.item()
                    total_acc += (predicted == targets).float().sum().item() / targets.numel()
        except KeyboardInterrupt:
            # 异常情况关闭
            proc.close()
        proc.close()
        return total_loss / len(loader), 100.0 * total_acc / len(loader)

    def test(self, loader: DataLoader) -> Tuple[float, float]:
        """Evaluate model using batches from loader and return accuracy and loss."""
        with torch.no_grad():
            total_loss, total_acc = 0.0, 0.0
            self.model.eval()
            for _, (inputs, targets) in tqdm(enumerate(loader), total=len(loader), desc='Testing '):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                
                _, predicted = torch.max(outputs, 1)
                
                total_loss += loss.item()
                total_acc += (predicted == targets).float().sum().item() / targets.numel()
        return total_loss / len(loader), 100.0 * total_acc / len(loader)   
    def predict(self, loader: DataLoader):
        results = []
        for inputs, targets in loader:
            inputs = inputs.to(self.device)
            outputs = self.model(inputs)
            _, predicted = torch.max(outputs, 1)
            results.append(predicted)
        return results
def run_epochs_for_loop(
        trainer: Trainer,
        epochs: int,
        train_loader: DataLoader,
        test_loader: DataLoader,
        scheduler: ReduceLROnPlateau = None):
    """Run train + evaluation loop for specified epochs.

    Save checkpoint to specified save folder when better optimum is found.
    If LR scheduler is specified, change LR accordingly.
    """
    for epoch in range(epochs):
        (train_loss, train_acc) = trainer.train(train_loader)
        (test_loss, test_acc) = trainer.test(test_loader)
        print()
        print("Epoch %d: TrainLoss %f \t TrainAcc %f" % (epoch, train_loss, train_acc))
        print("Epoch %d: TestLoss %f \t TestAcc %f" % (epoch, test_loss, test_acc))
        # 动态更新学习率
        if scheduler is not None:
            scheduler.step(test_acc)

In [ ]:
data_dir = "./AI研习社_鸟类识别比赛数据集"
selection = ["train_set", "val_set"]
# pretrained_data_dir = "./imagenet_class_index.json"
train_labels = os.path.join(data_dir, "train_pname_to_index.csv")
valid_labels = os.path.join(data_dir, "val_pname_to_index.csv")


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)


class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, data_label_path, data_transform, data_loader=default_loader):
        """

        :param data_path: 要读取的文件的路径
        :param data_label_path: 标签数据的路径
        :param data_transform: 数据变换模式
        :param data_loader: 加载方法
        """
        # 在label文件中注意不要加上标签
        df = pd.read_csv(data_label_path, header=None)
        self.data_loader = data_loader
        self.data_transform = data_transform
        self.data_path = data_path
        # 获取文件夹下的全部图片名
        # self.img_names = os.listdir(os.getcwd())
        # self.labels = [" ".join(img_name.split(".")[1].split("_")[:-1]) for img_name in self.img_names]
        self.img_names = list(df[0])
        self.labels = list(df[1])

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, item):
        img_name = self.img_names[item]
        img_path = os.path.join(self.data_path, img_name)
        label = self.labels[item]
        img = self.data_loader(img_path)
        try:
            img = self.data_transform(img)
            return img, label-1
        except:
            raise Exception("cannot transform image: {}".format(img_name))


def show_raw_data():
    pass


def show_dataset():
    pass


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using gpu: %s" % torch.cuda.is_available())

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    formats = {
        'train_set': [
            transforms.Resize(448),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(5),
            transforms.RandomCrop(448),
            transforms.ToTensor(),
            normalize,
        ],
        'val_set': [
            transforms.Resize(448),
            transforms.CenterCrop(448),
            transforms.ToTensor(),
            normalize,
        ],
    }
    data_label_paths = {
        "train_set": train_labels,
        "val_set": valid_labels,
    }
    # train_format = formats["train"]
    # valid_format = formats["valid"]
    data_sets = {}
    # for one in selection:
    #     data_sets[one] = datasets.ImageFolder(
    #         os.path.join(data_dir, one),
    #         transforms.Compose(formats[one]),
    #     )
    for one in selection:
        data_sets[one] = CustomDataset(os.path.join(data_dir, one),
                                       data_label_paths[one],
                                       transforms.Compose(formats[one]))
    data_loader = {
        "train":
            torch.utils.data.DataLoader(
                data_sets["train_set"],
                batch_size=32,
                shuffle=True,
                num_workers=4
            ),

        "valid": torch.utils.data.DataLoader(
                data_sets["val_set"],
                batch_size=5,
                shuffle=False,
                num_workers=4
            ),
    }
    model = BilinearModel(num_classes=200)
    criterion = nn.CrossEntropyLoss()
    # criterion = nn.NLLLoss()
    model.to(device)
    criterion.to(device)
    lr = 0.001
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr=lr)

    trainer = Trainer(model, criterion, optimizer, device)
    reduceLROnPlateau = ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.1,
        patience=10,
        verbose=True,
        threshold=1e-4,
    )
    # 预训练全连接层
    for param in model.features.parameters():
        param.requires_grad = False
    run_epochs_for_loop(trainer, 20, data_loader["train"], data_loader["valid"], reduceLROnPlateau)
    
    # 整体训练
    for param in model.features.parameters():
        param.requires_grad = True
    reduceLROnPlateau = ReduceLROnPlateau(optimizer, mode='max')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    run_epochs_for_loop(trainer, 5, data_loader["train"], data_loader["valid"], reduceLROnPlateau)


Using gpu: True


Training:   2%|▏         | 6/258 [00:05<04:07,  1.02it/s]

In [16]:
# 修改test_set为test，增加一个test文件夹嵌套
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
trans = transforms.Compose([
    transforms.Resize((448, 448)),
    # transforms.CenterCrop(448),
    transforms.ToTensor(),
    normalize,
])
predict_sets = torchvision.datasets.ImageFolder(os.path.join(data_dir, "test"), transform=trans)
test_loader = torch.utils.data.DataLoader(
            predict_sets,
            batch_size=1, # 一次一张一张的预测
            shuffle=False,
            num_workers=4
        )
tt = Trainer(model, criterion, optimizer, device)
predictions = tt.predict(test_loader)
answer = []
for index, cls in enumerate(predictions):
      # print(predictions[index])
      # break
      path = predict_sets.imgs[index][0]
      l = path.split("/")
      img_name = l[-1]
      answer.append((img_name, int(predictions[index])+1))
answer = sorted(answer, key=lambda x: int(x[0].split(".")[0]))
import csv
with open('test.csv','w', newline="")as f:
    writer = csv.writer(f)
    for one in answer:

      writer.writerow([one[0], one[1]])